<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Weekly newsletter based on issue/PR activity
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/GitHub_weekly_newsletter_based_on_issue_PR_activity.ipynb.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #tool #naas_drivers #naas #scheduler #asset #snippet #automation #ai #newsletter

**Author:** [Suhas B](https://www.linkedin.com/in/suhasbrao/)

The notebook allows you to create a weekly newsletter based on issue/PR activity for a GitHub repository.

## Input

### Import library

In [1]:
from naas_drivers import github
import naas
import markdown2
from IPython.core.display import display, HTML
import datetime
import pandas as pd

import requests
from urllib.parse import urlencode

### Variables

Before running the below cell ensure you have added a GitHub access token for repository using naas 
by running **naas.secret.add(name="API_NAME", secret="API_KEY")** and delete that cell.

You can find more info [here](https://docs.naas.ai/features/secret#add-or-edit-secret)

In [2]:
# naas.secret.list()
GITHUB_TOKEN = naas.secret.get(name="GITHUB_TOKEN")
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"
repo_name = "awesome-notebooks"

In [3]:
no_activities_to_show = 6
no_past_days = 7
no_issues_created = 0
no_issues_closed = 0

In [4]:
email_receivers = ["raos04567@gmail.com"] #list of email_receivers

# EMAIL_FROM = None
EMAIL_SUBJECT = "✉️ Weekly Repository activity "

# Markdown template created on your local env
EMAIL_CONTENT_MD = "email_content.md"

current_date = datetime.date.today()
previous_date = datetime.datetime.today() - datetime.timedelta(days= no_past_days)
previous_date = previous_date.date()
# print(previous_date)

### Schedule your notebook

Scheduling notebook to run At 12:00 on Sunday. You can modify it if you want 

In [5]:
# Schedule your notebook 
naas.scheduler.add(cron="0 12 * * 0")

#-> Uncomment the line below to remove your scheduler
# naas.scheduler.delete()

👌 Well done! Your Notebook has been sent to production.

⏰ It will be scheduled "At 12:00 every Sunday" (more on the syntax on https://crontab.guru/).

Ps: to remove the "Scheduler", just replace .add by .delete


## Model

### Get data from GitHub about issues and PR 

In [6]:
df_issues = github.connect(GITHUB_TOKEN).repos.get_issues(repo_url)
# df_pulls = github.connect(GITHUB_TOKEN).repos.get_pulls(repo_url)

In [29]:
def get_open_PRs(repo_url):
        """
        Return an dataframe object with 15 columns:
        - ID                      int64
        - ISSUE_URL               object
        - PR_NUMBER               int64
        - PR_STATE                object
        - TITLE                   object
        - FIRST_CREATED_DATE      object
        - FIRST_CREATED_TIME      object
        - LAST_UPDATED_DATE       object
        - LAST_UPDATED_TIME       object
        - COMMITS_URL             object
        - REVIEW_COMMENTS_URL     object
        - ISSUE_COMMENTS_URL      object
        - ASSIGNEES               object
        - REQUESTED_REVIEWERS     object
        - PR_ACTIVITY             object
        Parameters
        ----------
        repository: str:
            Repository url from Github.
            Example : "https://github.com/jupyter-naas/awesome-notebooks"
        """
        # Get organisation and repository from url
        git_obj = github.connect(GITHUB_TOKEN)
        repository = git_obj.get_repository_url(repo_url)
        
        df = pd.DataFrame()
        page = 1
        while True:
            params = {
                "per_page": "100",
                "page": page,
            }
            url = f"https://api.github.com/repos/{repository}/pulls?&{urlencode(params, safe='(),')}"
            res = requests.get(url, headers=git_obj.headers)
            try:
                res.raise_for_status()
            except requests.HTTPError as e:
                raise(e)
            res_json = res.json()
            if len(res_json) == 0:
                break

            for idx, r in enumerate(res_json):
                if r.get('state') == 'open':
                    df.loc[idx, 'id'] = r.get('id')
                    df.loc[idx, 'issue_url'] = r.get('html_url')
                    df.loc[idx, 'PR_number'] = r.get('number')
                    df.loc[idx, 'PR_state'] = 'open'
                    df.loc[idx, 'Title'] = r.get('title')
                    
                    df.loc[idx, 'issue_ref_title'] = r.get('head').get('ref')
                    
                    df.loc[idx, 'first_created_date'] = r.get('created_at').strip('Z').split('T')[0]
                    df.loc[idx, 'first_created_time'] = r.get('created_at').strip('Z').split('T')[-1]
                    df.loc[idx, 'last_updated_date'] = r.get('updated_at').strip('Z').split('T')[0]
                    df.loc[idx, 'last_updated_time'] = r.get('updated_at').strip('Z').split('T')[-1]

                    df.loc[idx, 'commits_url'] = r.get('commits_url')
                    df.loc[idx, 'review_comments_url'] = r.get('review_comments_url')
                    df.loc[idx, 'issue_comments_url'] = r.get('comments_url')

                    assignees_lst, reviewers_lst=[],[]
                    assignees_profile = []
                    for assignee in r.get('assignees'):
                        assignees_lst.append(assignee.get('login'))
                        assignees_profile.append(assignee.get('html_url'))
                    for reviewer in r.get('requested_reviewers'):
                        reviewers_lst.append(reviewer.get('login'))

                    if assignees_lst==[]:
                        df.loc[idx, 'assignees'] = 'None'
                        df.loc[idx, 'assignees_profile'] = 'None'
                    elif assignees_lst:
                        df.loc[idx, 'assignees'] = ", ".join(assignees_lst)
                        df.loc[idx, 'assignees_profile'] = ", ".join(assignees_profile)

                    if reviewers_lst==[]:
                        df.loc[idx, 'requested_reviewers'] = 'None'
                    elif reviewers_lst:
                        df.loc[idx, 'requested_reviewers'] = ", ".join(reviewers_lst)

                    date_format = "%Y-%m-%d"
                    delta = datetime.datetime.now() - datetime.datetime.strptime(df.loc[idx, 'last_updated_date'], date_format)
                    df.loc[idx, 'PR_activity'] = f'No activity since {delta.days} days'

                df['PR_number'] = df.PR_number.astype('int')
                df.id = df.id.astype('int')

            page+=1

        return df

In [30]:
df_open_PRS = get_open_PRs(repo_url)
df_open_PRS.head(5)
print(len(df_open_PRS))

9


In [9]:
def get_closed_issues(repo_url):
    '''
    This function retrives the closed issues of a repository and returns a 
    dataframe with the following columns:
    This function retrives the closed PRs of a repository and returns a 
    dataframe with the following columns:
    link_to_the_issue	
    issue_number	
    issue_title	
    issue_state	
    issue_assignees	
    last_created_date	
    last_created_time	
    last_updated_date	
    last_updated_time	
    link_to_the_pr	
    linked_pr_state	
    PR_activity

    Parameters
    ----------
    repository: str:
        Repository url from Github.
        Example : "https://github.com/jupyter-naas/awesome-notebooks"
    '''
    
    git_obj = github.connect(GITHUB_TOKEN)
    repository = git_obj.get_repository_url(repo_url)
    print(repository)
    
    
    df = pd.DataFrame()
    
    page, idx = 1, 0
    
    while True:
        
        params = {
                "per_page": "100",
                "page": page,
            }
        
        url = f"https://api.github.com/repos/{repository}/issues?state=closed&{urlencode(params, safe='(),')}"
        res = requests.get(url, headers=git_obj.headers)
    
        try:
            res.raise_for_status()
        except requests.HTTPError as e:
            raise(e)
        
        res_json = res.json()
        if len(res_json) == 0:
            break
                
        
        for issue in res_json:
            df.loc[idx,'link_to_the_issue'], df.loc[idx, 'issue_number'] = issue['html_url'], issue['number']
            df.loc[idx, 'issue_title'], df.loc[idx, 'issue_state'] = issue['title'], issue['state']
#             df.loc[idx, 'issue_id'] = issue['id']
        
            
            assigned=[]
            for assignee in issue['assignees']:
                assigned.append(assignee.get('login'))
            if assigned==[]:
                df.loc[idx, 'issue_assignees'] = 'None'
            else:
                df.loc[idx, 'issue_assignees'] = ", ".join(assigned)
            
            df.loc[idx, 'last_created_date'] = issue.get('created_at').strip('Z').split('T')[0]
            df.loc[idx, 'last_created_time'] = issue.get('created_at').strip('Z').split('T')[-1]
            df.loc[idx, 'last_updated_date'] = issue.get('updated_at').strip('Z').split('T')[0]
            df.loc[idx, 'last_updated_time'] = issue.get('updated_at').strip('Z').split('T')[-1]
            
            
            try:
#                 print(issue.get('pull_request')['url'])
                df.loc[idx, "link_to_the_pr"] = issue.get('pull_request')['html_url']
                pr = requests.get(issue.get('pull_request')['url'], headers= git_obj.headers).json()
                
                df.loc[idx, 'linked_pr_state'] = pr.get('state')

                date_format = "%Y-%m-%d"
                delta = datetime.datetime.now() - datetime.datetime.strptime(df.loc[idx, 'last_updated_date'], date_format)
                df.loc[idx, 'PR_activity'] = f'No activity since {delta.days} days'

            except Exception as e:
                #print(e)
                df.loc[idx, 'linked_pr_state'] = 'None'
                df.loc[idx, 'PR_activity'] = 'None'
            
            idx +=1
        
        page+=1
        

    return df

# print(len(get_closed_issues(repo_url)))

In [10]:
df_closed_issues = get_closed_issues(repo_url)

jupyter-naas/awesome-notebooks


In [31]:
def get_closed_PRs(repo_url):
    '''
    This function retrives the closed PRs of a repository and returns a 
    dataframe with the following columns:
        - ID                      int64
        - ISSUE_URL               object
        - PR_NUMBER               int64
        - PR_STATE                object
        - TITLE                   object
        - FIRST_CREATED_DATE      object
        - FIRST_CREATED_TIME      object
        - LAST_UPDATED_DATE       object
        - LAST_UPDATED_TIME       object
        - COMMITS_URL             object
        - REVIEW_COMMENTS_URL     object
        - ISSUE_COMMENTS_URL      object
        - ASSIGNEES               object
        - REQUESTED_REVIEWERS     object
        - REVIEWERS_PROFILE       object
        - PR_ACTIVITY             object
        Parameters
        ----------
        repository: str:
            Repository url from Github.
            Example : "https://github.com/jupyter-naas/awesome-notebooks"
    '''
    
    git_obj = github.connect(GITHUB_TOKEN)
    repository = git_obj.get_repository_url(repo_url)
    
    print(repository)
    
    df = pd.DataFrame()
    
    page, idx = 1, 0
#     page = 1
    while True:
            params = {
                "per_page": "200",
                "page": page,
            }
            url = f"https://api.github.com/repos/{repository}/pulls?state=closed&{urlencode(params, safe='(),')}"
            res = requests.get(url, headers=git_obj.headers)
            try:
                res.raise_for_status()
            except requests.HTTPError as e:
                raise(e)
            res_json = res.json()
            if len(res_json) == 0:
                break

            for _,r in enumerate(res_json):
#                 if r.get('state') == 'closed':
                    df.loc[idx, 'id'] = r.get('id')
                    df.loc[idx, 'issue_url'] = r.get('html_url')
                    df.loc[idx, 'PR_number'] = r.get('number')
                    df.loc[idx, 'PR_state'] = 'open'
                    df.loc[idx, 'Title'] = r.get('title')

                    df.loc[idx, 'first_created_date'] = r.get('created_at').strip('Z').split('T')[0]
                    df.loc[idx, 'first_created_time'] = r.get('created_at').strip('Z').split('T')[-1]
                    df.loc[idx, 'last_updated_date'] = r.get('updated_at').strip('Z').split('T')[0]
                    df.loc[idx, 'last_updated_time'] = r.get('updated_at').strip('Z').split('T')[-1]

                    df.loc[idx, 'commits_url'] = r.get('commits_url')
                    df.loc[idx, 'review_comments_url'] = r.get('review_comments_url')
                    df.loc[idx, 'issue_comments_url'] = r.get('comments_url')

                    assignees_lst, reviewers_lst=[],[]
                    reviewers_profile = []
                    for assignee in r.get('assignees'):
                        assignees_lst.append(assignee.get('login'))
                    for reviewer in r.get('requested_reviewers'):
                        reviewers_lst.append(reviewer.get('login'))
                        reviewers_profile.append(reviewer.get('html_url'))

                    if assignees_lst==[]:
                        df.loc[idx, 'assignees'] = 'None'
                    elif assignees_lst:
                        df.loc[idx, 'assignees'] = ", ".join(assignees_lst)

                    if reviewers_lst==[]:
                        df.loc[idx, 'requested_reviewers'] = 'None'
                        df.loc[idx, 'reviewers_profile'] = 'None'
                    elif reviewers_lst:
                        df.loc[idx, 'requested_reviewers'] = ", ".join(reviewers_lst)
                        df.loc[idx, 'reviewers_profile'] = ", ".join(reviewers_profile)

                    date_format = "%Y-%m-%d"
                    delta = datetime.datetime.now() - datetime.datetime.strptime(df.loc[idx, 'last_updated_date'], date_format)
                    df.loc[idx, 'PR_activity'] = f'No activity since {delta.days} days'

                    df['PR_number'] = df.PR_number.astype('int')
                    df.id = df.id.astype('int')
                    
                    idx +=1

            page+=1

    return df

# print(len(get_closed_issues(repo_url)))

In [32]:
df_closed_prs = get_closed_PRs(repo_url)

jupyter-naas/awesome-notebooks


In [33]:
# Get the latest issues between previous_date and current_date
# This is acheived by using a mask as shown below

mask_for_open_issues = (df_issues["last_created_date"] >= str(previous_date)) & \
                        (df_issues["last_created_date"] <= str(current_date))
# mask.head(5)

mask_for_closed_issues = (df_closed_issues["last_created_date"] >= str(previous_date)) & \
                        (df_closed_issues["last_created_date"] <= str(current_date))


mask_for_closed_PRs = (df_closed_prs["last_updated_date"] >= str(previous_date)) & \
                      (df_closed_prs["last_updated_date"] <= str(current_date))

mask_for_open_PRs = (df_open_PRS["last_updated_date"] >= str(previous_date)) & \
                      (df_open_PRS["last_updated_date"] <= str(current_date))


df_new_issues = df_issues.loc[mask_for_open_issues]
df_closed_issues_needed = df_closed_issues.loc[mask_for_closed_issues]
df_closed_PRs_needed = df_closed_prs.loc[mask_for_closed_PRs]
df_open_PRs_needed = df_open_PRS.loc[mask_for_open_PRs]

no_issues_created = len(df_new_issues)
no_issues_closed = len(df_closed_issues_needed)
no_PRs_closed = len(df_closed_PRs_needed)
no_PRs_open = len(df_open_PRs_needed)

## Create a Content for mail
This is later to converted to a HTML content for the mail. Modify these contents based on your needs.

In [45]:
# content = open(EMAIL_CONTENT_MD, "r").read()

content = f'''
Hello there,

Here is what happened over the past {no_past_days} days on **{repo_name}**:


### Issues created = {no_issues_created}
Here are few recent issues,

- [#{df_new_issues["issue_number"][0]} / {df_new_issues["issue_title"][0]}]({df_new_issues["link_to_the_issue"][0]})

- [#{df_new_issues["issue_number"][1]} / {df_new_issues["issue_title"][1]}]({df_new_issues["link_to_the_issue"][1]})

- [#{df_new_issues["issue_number"][2]} / {df_new_issues["issue_title"][2]}]({df_new_issues["link_to_the_issue"][2]})

- [#{df_new_issues["issue_number"][3]} / {df_new_issues["issue_title"][3]}]({df_new_issues["link_to_the_issue"][3]})

- [#{df_new_issues["issue_number"][4]} / {df_new_issues["issue_title"][4]}]({df_new_issues["link_to_the_issue"][4]})

- [#{df_new_issues["issue_number"][5]} / {df_new_issues["issue_title"][5]}]({df_new_issues["link_to_the_issue"][5]})

### Issues closed = {no_issues_closed}
Here are the recently closed ones,


- [#{df_closed_issues_needed["issue_number"][0]} \
    {df_closed_issues_needed["issue_title"][0]}]({df_closed_issues_needed["link_to_the_issue"][0]}),
    closed by PR-#[{df_closed_PRs_needed["PR_number"][0]}]({df_closed_PRs_needed["issue_url"][0]}) \
    by [@{df_closed_PRs_needed["requested_reviewers"][0]}]({df_closed_PRs_needed["reviewers_profile"][0]})

- [#{df_closed_issues_needed["issue_number"][1]} \
    {df_closed_issues_needed["issue_title"][1]}]({df_closed_issues_needed["link_to_the_issue"][1]}),
    closed by PR-#[{df_closed_PRs_needed["PR_number"][1]}]({df_closed_PRs_needed["issue_url"][1]}) \
    by [@{df_closed_PRs_needed["requested_reviewers"][1]}]({df_closed_PRs_needed["reviewers_profile"][1]})
    
- [#{df_closed_issues_needed["issue_number"][2]} \
    {df_closed_issues_needed["issue_title"][2]}]({df_closed_issues_needed["link_to_the_issue"][2]}),
    closed by PR-#[{df_closed_PRs_needed["PR_number"][2]}]({df_closed_PRs_needed["issue_url"][2]}) \
    by [@{df_closed_PRs_needed["requested_reviewers"][2]}]({df_closed_PRs_needed["reviewers_profile"][2]})
    
- [#{df_closed_issues_needed["issue_number"][3]} \
    {df_closed_issues_needed["issue_title"][3]}]({df_closed_issues_needed["link_to_the_issue"][3]}),
    closed by PR-#[{df_closed_PRs_needed["PR_number"][3]}]({df_closed_PRs_needed["issue_url"][3]}) \
    by [@{df_closed_PRs_needed["requested_reviewers"][3]}]({df_closed_PRs_needed["reviewers_profile"][3]})
    
- [#{df_closed_issues_needed["issue_number"][4]} \
    {df_closed_issues_needed["issue_title"][4]}]({df_closed_issues_needed["link_to_the_issue"][4]}),
    closed by PR-#[{df_closed_PRs_needed["PR_number"][4]}]({df_closed_PRs_needed["issue_url"][4]}) \
    by [@{df_closed_PRs_needed["requested_reviewers"][4]}]({df_closed_PRs_needed["reviewers_profile"][4]}) 


### PRs Open = {no_PRs_open}

Below are ther recent ones,

- PR [#{df_open_PRs_needed["PR_number"][0]}]({df_open_PRs_needed["issue_url"][0]}), solving Issue \
    "**{df_open_PRs_needed["issue_ref_title"][0]}**" from \
    @[{df_open_PRs_needed["assignees"][0]}]({df_open_PRs_needed["assignees_profile"][0]}) opened on \
    {df_open_PRs_needed["first_created_date"][0]}

- PR [#{df_open_PRs_needed["PR_number"][1]}]({df_open_PRs_needed["issue_url"][1]}), solving Issue \
    "**{df_open_PRs_needed["issue_ref_title"][1]}**" from \
    @[{df_open_PRs_needed["assignees"][1]}]({df_open_PRs_needed["assignees_profile"][1]}) opened on \
    {df_open_PRs_needed["first_created_date"][1]}
    
- PR [#{df_open_PRs_needed["PR_number"][2]}]({df_open_PRs_needed["issue_url"][2]}), solving Issue \
    "**{df_open_PRs_needed["issue_ref_title"][2]}**" from \
    @[{df_open_PRs_needed["assignees"][2]}]({df_open_PRs_needed["assignees_profile"][2]}) opened on \
    {df_open_PRs_needed["first_created_date"][2]}

- PR [#{df_open_PRs_needed["PR_number"][3]}]({df_open_PRs_needed["issue_url"][3]}), solving Issue \
    "**{df_open_PRs_needed["issue_ref_title"][3]}**" from \
    @[{df_open_PRs_needed["assignees"][3]}]({df_open_PRs_needed["assignees_profile"][3]}) opened on \
    {df_open_PRs_needed["first_created_date"][3]}

Check out the latest issues activity on GitHub

Cheers
'''

md = markdown2.markdown(content)
# email_content = replace_values(md)
display(HTML(md))

## Output

In [16]:
print("Nb issues:", len(df_issues))
df_issues.head(5)
# df_issues.loc[df_issues["issue_state"] == "closed"]

Nb issues: 577


,link_to_the_issue,issue_number,issue_title,issue_state,issue_id,issue_labels,issue_assignees,comments_till_date,last_created_date,last_created_time,last_updated_date,last_updated_time,comments,linked_pr_state,PR_activity
0,https://github.com/jupyter-naas/awesome-notebo...,1244,feat: [Issue# 1234] Kaggle - Download Datasets,open,1403494989,"enhancement, hacktoberfest",waqarg2001,4,2022-10-10,17:21:40,2022-10-12,05:46:35,"[""@waqarg2001! TY for your contribution 👍 \r\n...",open,No activity since 0 days
1,https://github.com/jupyter-naas/awesome-notebo...,1243,Adding generate weekly newsletter notebook,open,1403423514,None,SuhasBRao,0,2022-10-10,16:18:34,2022-10-10,16:32:38,No comments,open,No activity since 2 days
2,https://github.com/jupyter-naas/awesome-notebo...,1241,add: first commit,open,1403353533,None,unytics,1,2022-10-10,15:25:05,2022-10-11,11:04:55,['@unytics here we go '],open,No activity since 1 days
3,https://github.com/jupyter-naas/awesome-notebo...,1240,BigFunctions - Get sentiment score,open,1403334857,"enhancement, hacktoberfest",unytics,0,2022-10-10,15:12:14,2022-10-10,15:18:49,No comments,None,None
4,https://github.com/jupyter-naas/awesome-notebo...,1237,Naas - Send awesome-notebooks template list to...,open,1403099815,"enhancement, hacktoberfest",FlorentLvr,1,2022-10-10,12:31:01,2022-10-10,12:34:30,['This will personally help me monitor the tem...,None,None


In [19]:
print("Nb PRs:", len(df_open_PRS))
df_open_PRS.head(5)
# df_pulls["PR_activity"][1]

Nb PRs: 9


,id,issue_url,PR_number,PR_state,Title,issue_ref_title,first_created_date,first_created_time,last_updated_date,last_updated_time,commits_url,review_comments_url,issue_comments_url,assignees,requested_reviewers,PR_activity
0,1082318165,https://api.github.com/repos/jupyter-naas/awes...,1244,open,feat: [Issue# 1234] Kaggle - Download Datasets,kaggle-dataset,2022-10-10,17:21:40,2022-10-12,05:46:35,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,waqarg2001,"jravenel, FlorentLvr",No activity since 0 days
1,1082255474,https://api.github.com/repos/jupyter-naas/awes...,1243,open,Adding generate weekly newsletter notebook,1154-github-create-weekly-newsletter-based-on-...,2022-10-10,16:18:34,2022-10-10,16:32:38,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,SuhasBRao,FlorentLvr,No activity since 2 days
2,1082194362,https://api.github.com/repos/jupyter-naas/awes...,1241,open,add: first commit,1240-bigfunctions-get-sentiment-score,2022-10-10,15:25:05,2022-10-11,11:04:55,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,unytics,None,No activity since 1 days
3,1079416818,https://api.github.com/repos/jupyter-naas/awes...,1229,open,added python notebook to find phone number in ...,find_phone_number,2022-10-06,18:04:58,2022-10-11,10:51:54,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,anastazir,FlorentLvr,No activity since 1 days
4,1077599766,https://api.github.com/repos/jupyter-naas/awes...,1227,open,feat: added Cloud-Mercato VM pricing first commit,1226-cloud-mercato-compare-vms-pricing,2022-10-05,16:46:44,2022-10-11,11:05:47,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,ZuluPro,ZuluPro,No activity since 1 days


In [14]:
print("Nb PRs closed:", len(df_closed_prs))
df_closed_prs.head(6)

Nb PRs closed: 355


,id,issue_url,PR_number,PR_state,Title,first_created_date,first_created_time,last_updated_date,last_updated_time,commits_url,review_comments_url,issue_comments_url,assignees,requested_reviewers,reviewers_profile,PR_activity
0,1082687665,https://github.com/jupyter-naas/awesome-notebo...,1247,open,Python - Rename file #1233,2022-10-11,02:40:03,2022-10-11,11:34:03,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,rexdivakar,FlorentLvr,https://github.com/FlorentLvr,No activity since 1 days
1,1082461690,https://github.com/jupyter-naas/awesome-notebo...,1246,open,Changed for loop with list comprehension,2022-10-10,20:09:42,2022-10-11,15:34:18,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,None,None,NaN,No activity since 1 days
2,1082361379,https://github.com/jupyter-naas/awesome-notebo...,1245,open,[PR] added the template to reformat text that ...,2022-10-10,18:11:17,2022-10-11,11:27:18,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,None,None,NaN,No activity since 1 days
3,1082250400,https://github.com/jupyter-naas/awesome-notebo...,1242,open,Convert CSV URL file to Excel file format,2022-10-10,16:14:00,2022-10-11,11:16:47,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,Sophyia7,jravenel,https://github.com/jravenel,No activity since 1 days
4,1082157802,https://github.com/jupyter-naas/awesome-notebo...,1239,open,[Issue# 1234] Kaggle - Download Datasets,2022-10-10,14:56:52,2022-10-10,17:32:01,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,waqarg2001,jravenel,https://github.com/jravenel,No activity since 2 days
5,1082030512,https://github.com/jupyter-naas/awesome-notebo...,1238,open,added kaggle-download data.ipynb,2022-10-10,13:17:18,2022-10-10,18:44:46,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,https://api.github.com/repos/jupyter-naas/awes...,None,None,NaN,No activity since 2 days


In [15]:
print("No Issues closed:", len(df_closed_issues))
df_closed_issues.head(5)

No Issues closed: 651


,link_to_the_issue,issue_number,issue_title,issue_state,issue_assignees,last_created_date,last_created_time,last_updated_date,last_updated_time,link_to_the_pr,linked_pr_state,PR_activity
0,https://github.com/jupyter-naas/awesome-notebo...,1247.0,Python - Rename file #1233,closed,rexdivakar,2022-10-11,02:40:03,2022-10-11,11:34:03,https://github.com/jupyter-naas/awesome-notebo...,closed,No activity since 1 days
1,https://github.com/jupyter-naas/awesome-notebo...,1246.0,Changed for loop with list comprehension,closed,None,2022-10-10,20:09:42,2022-10-11,15:34:18,https://github.com/jupyter-naas/awesome-notebo...,closed,No activity since 1 days
2,https://github.com/jupyter-naas/awesome-notebo...,1245.0,[PR] added the template to reformat text that ...,closed,None,2022-10-10,18:11:17,2022-10-11,11:27:18,https://github.com/jupyter-naas/awesome-notebo...,closed,No activity since 1 days
3,https://github.com/jupyter-naas/awesome-notebo...,1242.0,Convert CSV URL file to Excel file format,closed,Sophyia7,2022-10-10,16:14:00,2022-10-11,11:16:47,https://github.com/jupyter-naas/awesome-notebo...,closed,No activity since 1 days
4,https://github.com/jupyter-naas/awesome-notebo...,1239.0,[Issue# 1234] Kaggle - Download Datasets,closed,waqarg2001,2022-10-10,14:56:52,2022-10-10,17:32:01,https://github.com/jupyter-naas/awesome-notebo...,closed,No activity since 2 days


### Send mail to receivers

In [46]:
for receiver in email_receivers:
    naas.notification.send(email_to=receiver,
                           subject=EMAIL_SUBJECT,
                           html=md)

👌 💌 Email has been sent successfully !
